In [2]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import seaborn as sns
import re
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
from scipy.stats import t

from chapters.chapter8 import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
theta = (t.ppf(0.75, 3) - t.ppf(0.25, 3)) / 1.34
theta

1.1416303404192403

In [4]:
obs = t.rvs(3, size=25)
obs

array([ 0.97442783, -0.37559688,  4.67457867,  0.93902427, -0.28586191,
        0.02619128, -0.37147975, -1.0917111 ,  1.48828415,  0.5258335 ,
       -0.40666186, -0.78900618,  0.02991144, -0.81607991, -0.38627618,
       -1.19756689,  0.73345944,  0.51085135, -4.08510571, -1.05274502,
       -0.2053558 ,  3.8510504 ,  2.1216446 , -0.26205937,  1.17686852])

In [5]:
statistic = lambda obs: (np.percentile(obs, 75) - np.percentile(obs, 25)) / 1.34
statistic(obs)

1.0042433830474906

In [11]:
data = {'lower': [], 'upper': [], 'type': []}

for i in range(1000):
    
    obs = pd.Series(t.rvs(3, size=25))
    
    nci = bootstrap_ci_normal(obs, statistic, 100, 0.05)    
    pivci = bootstrap_ci_pivot(obs, statistic, 100, 0.05)    
    perci = bootstrap_ci_percentile(obs, statistic, 100, 0.05)
    
    if i % 100 == 0:
        print(nci, pivci, perci)
    
    data.get('lower').append(nci.lower)    
    data.get('upper').append(nci.upper)
    data.get('type').append('normal')
    
    data.get('lower').append(pivci.lower)    
    data.get('upper').append(pivci.upper)
    data.get('type').append('pivot')
    
    data.get('lower').append(perci.lower)    
    data.get('upper').append(perci.upper)
    data.get('type').append('percentile')
    
df = pd.DataFrame(data)
df['contains_true'] = df.apply(lambda r: r.lower <= theta <= r.upper, axis=1)

#df.to_csv('../data/ch08q03.csv', index=False)

df.head()

CI(lower=0.5156639919152145, upper=1.7167599516444088) CI(lower=0.50928145601241681, upper=1.6958873701224797) CI(lower=0.60911819752546137, upper=1.7146732401098512)
CI(lower=0.45410857167372232, upper=1.6607720152176646) CI(lower=0.57279572848059956, upper=1.4535995749343953) CI(lower=0.54014743080611638, upper=1.7647738255667409)
CI(lower=0.60385196439111488, upper=2.0267615846572715) CI(lower=0.44406337611351798, upper=1.8036941670598468) CI(lower=0.82691938198853987, upper=1.8807297349587597)
CI(lower=0.31543281539107648, upper=1.4306422972329282) CI(lower=0.1446363178002632, upper=1.3344638907117967) CI(lower=0.43377991400451787, upper=1.7456737845072383)
CI(lower=-0.25938356924185846, upper=1.6910671291441635) CI(lower=-1.0234556540551818, upper=0.99568040971998006) CI(lower=0.42373786678618308, upper=2.4053288178856)
CI(lower=0.10557971324865545, upper=1.3203010453574411) CI(lower=-0.032268548076881842, upper=1.0878079996932903) CI(lower=0.22435652777561224, upper=1.21588811022

,lower,type,upper
0,0.515664,normal,1.716760
1,0.509281,pivot,1.695887
2,0.609118,percentile,1.714673
3,0.783856,normal,1.865298
4,0.613448,pivot,1.934928


In [19]:
df.groupby('type').contains_true.mean()

type
normal        0.947
percentile    0.957
pivot         0.814
Name: contains_true, dtype: float64